# Data Standardisation Pipeline

In this notebook, I detail the process of importing data into `crowdastro`.

## Input data sources

The input data sources are:

- Radio Galaxy Zoo
    - Subjects (`radio_subjects.json`)
    - Classifications (`radio_classifications.json`)
- ATLAS
    - Catalogue (`ATLASDR3_cmpcat_23July2015.dat`)
    - FITS images of the radio sky (`cdfs` & `elais`)
- SPITZER
    - SWIRE Catalogue (`SWIRE3_CDFS_cat_IRAC24_21Dec05.tbl`)
    - FITS images of the infrared sky (`cdfs` & `elais`)

Paths to these should be specified in `crowdastro.json`. Radio Galaxy Zoo data should be imported into a database in MongoDB, specified by `radio_galaxy_zoo_db` in `crowdastro.json`. Following is an example `crowdastro.json`:

```json
{
    "data_sources": {
        "atlas_catalogue": "data/ATLASDR3_cmpcat_23July2015.dat",
        "cdfs_fits": "data/cdfs",
        "elais_fits": "data/elais",
        "radio_galaxy_zoo_db": "radio",
        "swire_catalogue": "data/SWIRE3_CDFS_cat_IRAC24_21Dec05.tbl"
    },

    "mongo": {
        "host": "localhost",
        "port": 27017
    }
}
```